In [2]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import os
os.getcwd()
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [5]:
from tensorflow.keras.datasets import imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)
print(X_train[0]) # 문장에서 나온 각 단어를 인덱스 값으로 변환한 것(전처리가 끝난 상태)
print(y_train[0]) # 0: negative, 1: positive

17465344/17464789 [==============================] - 0s 0us/step
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


In [6]:
# 문장마다 길이가 다르므로 길이를 맞춰주기
# 문장마다 embedding layer를 통과하게 할 것이기 때문
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len) # 더 길면 자르고, 짧으면 0을 추가
X_test = pad_sequences(X_test, maxlen=max_len)

In [7]:
print(X_train[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [7]:
# 모델 생성-------------------------------------------

model = Sequential()
# 단어 임베딩 -> 5000개의 단어를 120차원으로 내보내겠다(정수 -> 120 dimension vector)
model.add(Embedding(5000, 120))
# LSTM
model.add(LSTM(120))
# 이진 분류(sigmoid)
model.add(Dense(1, activation='sigmoid'))
#------------------------------------------------------

In [8]:
# 모델 성능 향상---------------------------------------

# validation loss를 계속 보다가 5회 이상 loss가 증가하면, 과적합될 수 있으므로 학습을 조기 종료하겠다.
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# epoch를 반복하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트(the_best.h5)로 저장
# 정확도가 낮아지면 모델 버려라
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[early_stop, model_check])

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 0.4408 - acc: 0.7887
Epoch 00001: val_acc improved from -inf to 0.84736, saving model to the_best.h5
391/391 [==============================] - 521s 1s/step - loss: 0.4408 - acc: 0.7887 - val_loss: 0.3623 - val_acc: 0.8474
Epoch 2/10
391/391 [==============================] - ETA: 0s - loss: 0.2779 - acc: 0.8892
Epoch 00002: val_acc improved from 0.84736 to 0.87568, saving model to the_best.h5
391/391 [==============================] - 524s 1s/step - loss: 0.2779 - acc: 0.8892 - val_loss: 0.2991 - val_acc: 0.8757
Epoch 3/10
391/391 [==============================] - ETA: 0s - loss: 0.2259 - acc: 0.9136
Epoch 00003: val_acc did not improve from 0.87568
391/391 [==============================] - 521s 1s/step - loss: 0.2259 - acc: 0.9136 - val_loss: 0.3076 - val_acc: 0.8718
Epoch 4/10
391/391 [==============================] - ETA: 0s - loss: 0.1977 - acc: 0.9246
Epoch 00004: val_acc improved from 0.87568 to 0.87756, sa

In [9]:
# # 모델 불러오기
# from tensorflow.keras.models import load_model

# model = load_model('the_best.h5')

In [10]:
# 정확도 측정
print(model.evaluate(X_test, y_test))

782/782 [==============================] - 100s 128ms/step - loss: 0.3229 - acc: 0.8776
[0.32294711470603943, 0.877560019493103]
